In [1]:
import pandas as pd
import numpy as np

import librosa
from librosa import display
import os
import tensorflow as tf
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
from sklearn.metrics import precision_score


2024-04-04 21:52:00.630803: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-04 21:52:01.347661: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
loaded_model = tf.keras.models.load_model('cnn_model.keras')

In [3]:
file = '2x1.wav'

In [4]:
feature_list = []
audio_path = os.path.join('./operations', file)
audio, sr = librosa.load(path=audio_path, sr=None)
audio = librosa.effects.time_stretch(y=audio, rate=len(audio)/sr)
# Calculate features
mels = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=1048, hop_length=512)
mels_db = librosa.power_to_db(S=mels, ref=1.0)
feature_list.append(mels_db)


# Determine the maximum length of the second dimension among all features
max_length = max(feature.shape[1] for feature in feature_list)

# Pad or truncate each feature to have the same length as max_length
for i, feature in enumerate(feature_list):
    pad_width = max_length - feature.shape[1]
    # Pad or truncate along the second dimension
    if pad_width > 0:
        feature_list[i] = np.pad(feature, ((0, 0), (0, pad_width)), mode='constant')
    elif pad_width < 0:
        feature_list[i] = feature[:, :max_length]

# Now, all features have the same shape, so you can reshape them
feature_list = [feature.reshape((128, -1, 1)) for feature in feature_list]

# Convert feature_list and label_list to NumPy arrays
features = np.array(feature_list)

In [5]:
y_predict = loaded_model.predict(features)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step


In [6]:
original_labels = np.argmax(y_predict, axis=1)

In [7]:
original_labels[0]

16